In [1]:
import pandas as pd
pd.set_option('display.max_columns', 200) #  max number of columns
import geopandas as gpd
import contextily as cx

In [3]:
df = pd.read_csv("data_files/aisdk-2024-02-18.csv", parse_dates=["# Timestamp"], index_col="# Timestamp")

/var/folders/r8/9_x5mqbs79gbpktlgmt3lyb40000gn/T/ipykernel_28938/2787948836.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv("data_files/aisdk-2024-02-18.csv", parse_dates=["# Timestamp"], index_col="# Timestamp")


In [8]:
from utils.data_prep import *

ModuleNotFoundError: No module named 'movingpandas'

In [ ]:
df.dtypes

In [ ]:
df.loc[(df['Ship type'] != "Undefined") & (df['Ship type'] != "Other")].head(25)

In [ ]:
def geo_transform(MMSI: int):
    helgoland = df.copy().loc[df["MMSI"] == MMSI]
    # map timestamp into datetime object
    helgoland["# Timestamp"] = pd.to_datetime(helgoland["# Timestamp"])
    helgoland.set_index("# Timestamp", inplace=True)

    helgoland = helgoland[['MMSI', 'Latitude', 'Longitude']]
    helgoland.resample('10T').mean() # resample data frequency to 10 minutes

    helgoland = gpd.GeoDataFrame(helgoland, geometry=gpd.points_from_xy(helgoland.Longitude, helgoland.Latitude))
    return helgoland

In [ ]:
ship1 = geo_transform(MMSI=538005405)
ship1

In [ ]:
ship1.crs = "EPSG:3857"

In [ ]:
ship1[1:]

In [ ]:
ship1

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(1, figsize=(10, 10))
ship1[1:].plot(alpha=0.5, ax=ax)

ax.set_axis_off()
cx.add_basemap(ax, crs=ship1.crs, source=cx.providers.CartoDB.Positron)